In [21]:
%matplotlib inline

import sys
sys.path.append('../lib')

*We spoke about the basic operations like **binding** and **bundling**, but the **sub-symbolic** level allows us to do other operations which are "composite" and have no equivalent in non VSA architecture, that is because we can use SDP Algebra as I mentioned.*


## Analogy mapping

Humans use analogy all the time, it is integral part of intelligence. What if I told you we can mimic a proto form of analogy on syb-symbolic level via vector math. Nooo waaay ! .... way!

Here is the idea :

$ analogy\_map = relation * reverse\_relation $

if we do the algebraic manipulation :

$ analogy\_map * relation = reverse\_relation $

because the **bind** operation is two way street. 

Now we can use the first equation for a **training** operation and the second as **predicting/testing** operation.
The important part is that we can do this operation on a whole **structure** too, not just on single term.
Plus this is **one-shot** learning, something you can't do with Neuro-networks.

Let concretize the idea, the relations we will pick are **"above"** and correspondingly the reverse **"below"**. We will train **"analogical map"** that will virtually "swap" the position of two relations i.e. make one relation transform to the other. 
Here is what we have :

-- *circle above a square* --

$ cas = above + a\_role1 * circle + a\_role2 * square $


Let see the reverse :

-- *square below a circle* --

$ sbc = below + b\_role1 * square + b\_role2 * circle $


Now we learn the mapping (one-shot learning) :

$ map = cas * sbc $

and then we can apply it to unknown (not trained with) objects. Lets define them, so that we can do the comparison.


$ sat = above + a\_role1 * star + a\_role2 * triangle $

$ tbs = below + b\_role1 * triangle + b\_role2 * star $

So if we want to transform the "triangle" above the "star' =to=> 'star' below 'triangle', we will bind it with the learned map. Of course it is approximate match. That is vectors for you.

`tbs <=~=> map * sat`

`sat <=~=> map * tbs`

The operation works both ways. 

If you want it to work only in one direction you should use permutation-bind when defining (have not tested it).

These kind of operations are also called **holistic transformations** where one compositional structure is mapped onto another compositional structure without having to first decompose the source representation into its components

Ok now that we know the theory, let's try it.

In [22]:
from bi import *
from lexicon import *

x = lex()
x.add_items(['above', 'below', 'a1', 'a2', 'b1', 'b2', 'circle', 'square', 'star', 'triangle'])

We should use sdp.bundle(), instead of **+**, because of the even-oddity I mentioned earlier /adds too much noise and it may not work/.

In [23]:
#training data
cas = sdp.bundle([ x['above'], x['a1'] * x['circle'],   x['a2'] * x['square'] ])
sbc = sdp.bundle([ x['below'], x['b1'] * x['square'],   x['b2'] * x['circle'] ])

#novel/testing data
sat = sdp.bundle([ x['above'], x['a1'] * x['star'],     x['a2'] * x['triangle'] ])
tbs = sdp.bundle([ x['below'], x['b1'] * x['triangle'], x['b2'] * x['star'] ])

#partially closer to the training data : 'square' is used in both at the same position
sas = sdp.bundle([ x['above'], x['a1'] * x['star'],     x['a2'] * x['square'] ])
sbs = sdp.bundle([ x['below'], x['b1'] * x['square'],   x['b2'] * x['star'] ])

#misplased 'square' i.e. novel data
sas2 = sdp.bundle([ x['above'], x['a1'] * x['square'],     x['a2'] * x['star'] ])
sbs2 = sdp.bundle([ x['below'], x['b1'] * x['star'],   x['b2'] * x['square'] ])

Lets learn the map (one-shot learning) :

In [24]:
M = cas * sbc

now test it against the training example (measuring distance, btw). Seems OK.

In [25]:
(M * cas) % sbc

0

Now lets try against the test structure, which we did not use in training : 

In [26]:
(M * sat) % tbs

3721

In our case as we mentioned the mapping is bi-directional, so :

In [27]:
(M * tbs) % sat

3721

The same using sdp.dist()

In [28]:
sdp.dist((M * sat), tbs)

3721

The distance is ~38%, but we said that a distance below ~42% means that the symbols are similar (sdp.true_thresh).

In [29]:
sdp.sim((M * sat), tbs)

True

Lets also test it with data that partially match with the training data :

In [33]:
sdp.dist((M * sas), sbs)

2483

The distance as expected is smaller ~25%. 

Once more but this time 'square' is placed in the 'wrong' position i.e. it will represent again in a sense novell data, ergo distance again becomes ~37%.

In [31]:
sdp.dist((M * sas2), sbs2)

3767

And finally lets test against random SDP, to see if they are orthogonal.

In [34]:
sdp.dist((M * sat), sdp.rand())

4972

That ends our exploration of analogy mapping, but I suppose you can guess that there is big unexplored territory in the sub-symbolic space for other ways of building composite-operations on structures, if you follow the ideas described in this article. 

You can see more detailed test at **'test/analogy_mapping.py'**.